In [1]:
# My Standard Spark Session!

# Python libraries:
import os
import sys
import re
from dateutil import parser
# import datetime
from datetime import datetime
from datetime import date
import builtins
import json
import functools
import operator
from itertools import product

# Numpy & Pandas!
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:20,.2f}'.format
pd.options.display.max_columns = None

# Spark!
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *
from pyspark.sql import SparkSession, Row


spark = SparkSession.builder.appName("myapp").getOrCreate()

#     spark = SparkSession.builder.master("yarn")\
#     .config("spark.executor.instances", "32")\
#     .config("spark.executor.cores", "4")\
#     .config("spark.executor.memory", "4G")\
#     .config("spark.driver.memory", "4G")\
#     .config("spark.executor.memoryOverhead","4G")\
#     .config("spark.yarn.queue","Medium")\
#     .appName("myapp")\
#     .getOrCreate()

sc = spark.sparkContext
spark.conf.set("spark.sql.sources.partitionColumnTypeInference.enabled", "false")
spark.conf.set("spark.debug.maxToStringFields","true")

In [2]:
%load_ext autoreload
%autoreload 2
# The autoreload extension is already loaded. To reload it, use:
#  %reload_ext autoreload


# mylib:
my_library = os.path.expanduser('~/.myconfigs')
my_spark = os.path.expanduser('~/spark2_dfanalysis')
sys.path.append(my_library)
sys.path.append(my_spark)


from shared.app_context import *
from builder.DataFrameBuild import *

ctx = ApplicationContext("Dev-Job")

DFB = DataFrameBuild(ctx.spark)

print("%16s  %s" % ("Python Version:",sys.version))
print("%16s  %s" % ("Python Path:",os.path.dirname(sys.executable)))
print("%16s  %s" % ("My Python Libs:",my_library))
print("%16s  %s" % ("My Spark Dir:",my_spark))
print("%16s  %s" % ("My Spark Ctx:",ctx.spark))
# print(ctx.spark)
# print(os.listdir(my_spark))
# print(sys.path)
# print("\n")

 Python Version:  3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)]
    Python Path:  C:\Users\d810216\AppData\Local\conda\conda\envs\my_root
 My Python Libs:  C:\Users\d810216/.myconfigs
   My Spark Dir:  C:\Users\d810216/spark2_dfanalysis
   My Spark Ctx:  <pyspark.sql.session.SparkSession object at 0x0000022471C02400>


In [9]:
# 6 columns: strings, ints, float
import random

num = 100000

lst_months = ['January','Feburary','March','April','May','June',
              'July','August','September','October','November','December']

indices = [int(x) for x in np.linspace(0,num-1,num)]
# print(indices)

years = [random.randint(1900,2019) for x in range(num)]
# print(years)

ages = [(2019 - x) for x in years]
# print(ages)

months = [random.choice(lst_months) for x in range(num)]
# print(months)

worth = [random.random() * 100000 for x in range(num)]
# print(worth)

lst_names = ['sarah','bill','steve','mary','alyssa','brian','elizabeth','josh','ryan','katie','connor',
             'erica','lisa','doug','stacie','chris','gary','tom','ellen','paula']
names = [random.choice(lst_names) for x in range(num)]


df = DFB.arrays_to_dataframe([indices,names,months,years,ages,worth],
                             ['index','name','month','year','age','net_worth'])


df.limit(4).toPandas()

,index,name,month,year,age,net_worth
0,0,connor,Feburary,1923,96,"32,974.81"
1,1,elizabeth,Feburary,1943,76,"4,068.52"
2,2,katie,October,1988,31,"13,206.11"
3,3,alyssa,May,1900,119,"89,850.34"


# Working Example:

## Let's categorize each sale as low, medium, or high
- low: <= 20,000
- medium: 20,000 < x < 34,000
- high: 34,000 <= 100,000

df.withColumn("